<a href="https://colab.research.google.com/github/Tushar0852/Project-perceptron/blob/main/project_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
class NeuronUnit:
    def __init__(self, num_inputs):
        self.weights = np.random.rand(num_inputs)
        self.bias = np.random.rand()
        self.lrate = 0.1
        self.value = 0.0
        self.delta = 0.0

In [14]:
class NeuralLayer:
    def __init__(self, num_neurons, num_inputs):
        self.neurons = [NeuronUnit(num_inputs) for _ in range(num_neurons)]
        self.length = num_neurons


In [15]:
class MultiLayerNetwork:
    def __init__(self, l_rate, transfer_function, transfer_function_derivative):
        self.l_rate = l_rate
        self.transfer_function = transfer_function
        self.transfer_function_derivative = transfer_function_derivative
        self.neural_layers = []

    def prepare_layer(self, num_neurons, num_inputs):
        layer = NeuralLayer(num_neurons, num_inputs)
        self.neural_layers.append(layer)

    def prepare_mlp_net(self, layers, l_rate, transfer_function, transfer_function_derivative):
        self.l_rate = l_rate
        self.transfer_function = transfer_function
        self.transfer_function_derivative = transfer_function_derivative

        self.neural_layers = []
        for i in range(len(layers)):
            num_neurons = layers[i]
            num_inputs = layers[i - 1] if i != 0 else 0
            self.prepare_layer(num_neurons, num_inputs)

        print("Complete Multilayer Perceptron init.")

In [16]:
def execute(mlp, pattern):
    # Set input values
    for i in range(len(pattern.features)):
        mlp.neural_layers[0].neurons[i].value = pattern.features[i]

    # Forward pass
    for k in range(1, len(mlp.neural_layers)):
        for i in range(mlp.neural_layers[k].length):
            nv = sum(mlp.neural_layers[k].neurons[i].weights[j] * mlp.neural_layers[k - 1].neurons[j].value
                     for j in range(mlp.neural_layers[k - 1].length))
            nv += mlp.neural_layers[k].neurons[i].bias
            mlp.neural_layers[k].neurons[i].value = mlp.transfer_function(nv)

    # Get output values
    return [neuron.value for neuron in mlp.neural_layers[-1].neurons]


In [17]:
def backpropagate(mlp, pattern, expected_output):
    no = execute(mlp, pattern)
    error = 0.0

    # Output layer
    for i in range(mlp.neural_layers[-1].length):
        e = expected_output[i] - no[i]
        mlp.neural_layers[-1].neurons[i].delta = e * mlp.transfer_function_derivative(no[i])

    # Hidden layers
    for k in range(len(mlp.neural_layers) - 2, -1, -1):
        for i in range(mlp.neural_layers[k].length):
            e = sum(mlp.neural_layers[k + 1].neurons[j].delta * mlp.neural_layers[k + 1].neurons[j].weights[i]
                    for j in range(mlp.neural_layers[k + 1].length))
            mlp.neural_layers[k].neurons[i].delta = e * mlp.transfer_function_derivative(mlp.neural_layers[k].neurons[i].value)

        # Update weights and biases
        for i in range(mlp.neural_layers[k + 1].length):
            for j in range(mlp.neural_layers[k].length):
                mlp.neural_layers[k + 1].neurons[i].weights[j] += mlp.l_rate * mlp.neural_layers[k + 1].neurons[i].delta * mlp.neural_layers[k].neurons[j].value
            mlp.neural_layers[k + 1].neurons[i].bias += mlp.l_rate * mlp.neural_layers[k + 1].neurons[i].delta

    # Calculate global error
    for i in range(len(expected_output)):
        error += abs(no[i] - expected_output[i])

    return error / len(expected_output)


In [18]:
def mlp_train(mlp, patterns, mapped, epochs):
    output = [0.0] * len(mapped)
    epoch = 0

    while epoch <= epochs:
        total_error = 0.0
        for pattern in patterns:
            for io in range(len(output)):
                output[io] = 0.0
            output[int(pattern.single_expectation)] = 1.0
            total_error += backpropagate(mlp, pattern, output)

        # Print the average error after each epoch
        avg_error = total_error / len(patterns)
        print(f"Epoch {epoch} - Average Error: {avg_error:.4f}")

        epoch += 1


In [19]:
def display_predictions(mlp, patterns):
    for i, pattern in enumerate(patterns):
        output = execute(mlp, pattern)
        print(f"Pattern {i} - Input: {pattern.features}, Predicted: {output}, Expected: {pattern.single_expectation}")


In [21]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))
